In [183]:
import tensorflow as tf

In [184]:
daily_sales_numbers = [21, 22, -108, 31, -2, 32, 654]

In [185]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

In [186]:
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [187]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-2
32
654


In [188]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [189]:
tf_dataset = tf_dataset.filter(lambda x:x>0)
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
31


In [190]:
tf_dataset = tf_dataset.map(lambda x: x*87)
for sales in tf_dataset.take(3):
    print(sales.numpy())

1827
1914
2697


In [191]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset:
    print(sales.numpy())

1914
1827
56898
2697
2784


In [192]:
for sales_batch in tf_dataset.batch(4):
    print(sales_batch.numpy())

[ 1914  1827  2697 56898]


[2784]


In [193]:
# all in one single line

In [194]:
tf_dataset2 = tf.data.Dataset.from_tensor_slices(daily_sales_numbers).filter(lambda x:x>0).map(lambda x: x*87).shuffle(2).batch(2)


for sales in tf_dataset:
    print(sales.numpy())

1827
2697
2784
1914
56898


In [195]:
image_ds = tf.data.Dataset.list_files('images/*/*', shuffle=False)

for file in image_ds.take(5):
    print(file.numpy())

b'images\\cat\\41QZSg8drOL.jpg'
b'images\\cat\\81v+HhDz51L.jpg'
b'images\\cat\\cat-1045782_640.jpg'
b'images\\cat\\cat-pet-animal-domestic-104827.jpeg_cs=srgb&dl=pexels-pixabay-104827.jpg'
b'images\\cat\\cat6-1744094950340-e44df3c1-dcf7-4164-8199-1182c8d838df-900x506.png'


In [196]:
image_ds = image_ds.shuffle(200)

for file in image_ds.take(5):
    print(file.numpy())

b'images\\cat\\io_ZGkgPmYBG95Xk-RBBd3W_v1_.png'
b'images\\cat\\pexels-photo-669015.jpeg_cs=srgb&dl=pexels-septimiu-lupea-208604-669015.jpg'
b'images\\dog\\download (8).jpeg'
b'images\\cat\\cat-1045782_640.jpg'
b'images\\dog\\download.jpeg'


In [197]:
class_names = ['cat',  'dog']

In [198]:
image_count = len(image_ds)

In [199]:
image_count

21

In [200]:
train_size = int(image_count* 0.8)

train_ds = image_ds.take(train_size)
test_ds = image_ds.skip(train_size)

In [201]:
def get_label(file_path):
   import os
   return tf.strings.split(file_path, os.path.sep)[-2]

In [205]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    # Use decode_jpeg instead of decode_image to get shape information
    img = tf.image.decode_jpeg(img, channels=3)  # Explicitly use JPEG decoder
    img = tf.image.resize(img, [128, 128])
    return img, label

In [206]:
# for t, c in train_ds.take(4):
#     print(t.numpy())

In [207]:
train_ds = train_ds.map(process_image)
for img, label in train_ds.take(3):
    print(label)
    print(img)

tf.Tensor(b'dog', shape=(), dtype=string)
tf.Tensor(
[[[54.445312  62.445312  13.4453125]
  [56.86592   64.86592   13.865921 ]
  [60.14453   69.14453   14.091461 ]
  ...
  [43.        32.        12.       ]
  [43.        32.        12.       ]
  [43.06813   33.18141   12.810318 ]]

 [[52.527344  60.527344  10.425781 ]
  [55.671875  63.671875  12.671875 ]
  [60.14453   69.14453   13.855469 ]
  ...
  [44.101562  33.101562  12.       ]
  [44.101562  33.101562  12.       ]
  [44.268265  34.381546  12.90889  ]]

 [[48.875     56.875      5.875    ]
  [53.75148   62.56546    9.9375   ]
  [59.359238  68.49191   11.234238 ]
  ...
  [46.835938  35.835938  13.8359375]
  [46.835938  35.835938  13.8359375]
  [46.495285  36.608566  14.237473 ]]

 ...

 [[68.723465  83.723465  16.887527 ]
  [68.826645  83.826645  16.990707 ]
  [68.929825  83.929825  17.093887 ]
  ...
  [77.08203   91.16406   28.917969 ]
  [77.08203   91.08203   29.18602  ]
  [77.82422   90.3094    29.77182  ]]

 [[64.28125   79.2812

In [208]:
def scale(image, label):
    return image/255, label

In [209]:
train_ds = train_ds.map(scale)
for image, label in train_ds.take(5):
    print(image.numpy()[0][0])
    print(label.numpy())

[0.07462087 0.10991498 0.00403263]
b'dog'
[0.847886   0.8282782  0.84396446]
b'dog'
[0.6195281  0.56462616 0.46266538]
b'cat'
[0.7294118  0.75686276 0.7882353 ]
b'dog'
[0.21351103 0.24488358 0.05272672]
b'dog'
